# **Sprint03: Detecção de Cáries / Análise de Imagens Odontológicas Panoramic Dental Dataset**

Este notebook demonstra o processo de:
1. Instalação e configuração da API do Kaggle no Google Colab.
2. Download e descompactação de um dataset de imagens (Panoramic Dental Dataset).
3. Carregamento e pré-processamento dos dados.
4. Construção, compilação e treinamento de um modelo de rede neural.
5. Avaliação e análise dos resultados obtidos.

As melhorias em relação a versões anteriores incluem uma melhor organização do código, adição de comentários explicativos e sugestões de extensões (como callbacks, visualização de métricas e análise de erros).


## 1. Instalação da Biblioteca Kaggle

Nesta etapa, instalamos a biblioteca `kaggle` no ambiente do Google Colab. Isso permite interagir diretamente com o Kaggle para baixar datasets, enviar submissões de competições, etc.


In [ ]:

!pip install kaggle

## 2. Upload da Chave da API do Kaggle

Aqui fazemos o upload do arquivo `kaggle.json`, que contém as credenciais de acesso à sua conta Kaggle.  


In [ ]:

from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"pedrolpsp","key":"e26a2092d79f521258efa7648bbc2443"}'}

## 3. Configuração do Diretório do Kaggle

Após o upload, precisamos mover o `kaggle.json` para o diretório padrão `~/.kaggle/` e ajustar as permissões para que a biblioteca Kaggle possa acessá-lo corretamente.


In [ ]:

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

## 4. Download do Dataset

Agora podemos baixar o dataset escolhido no Kaggle.  
**Exemplo**: `thunderpede/panoramic-dental-dataset`.


In [ ]:

!kaggle datasets download -d  thunderpede/panoramic-dental-dataset

Dataset URL: https://www.kaggle.com/datasets/thunderpede/panoramic-dental-dataset
License(s): other
100% 243M/244M [00:02<00:00, 129MB/s]
100% 244M/244M [00:02<00:00, 124MB/s]


## 5. Descompactar o Dataset

Por fim, descompactamos o arquivo `.zip` baixado para obtermos as pastas e arquivos de imagem.


In [ ]:

!unzip panoramic-dental-dataset.zip

Archive:  panoramic-dental-dataset.zip
  inflating: annotations/bboxes_caries/1008.txt  
  inflating: annotations/bboxes_caries/1009.txt  
  inflating: annotations/bboxes_caries/1016.txt  
  inflating: annotations/bboxes_caries/1018.txt  
  inflating: annotations/bboxes_caries/1026.txt  
  inflating: annotations/bboxes_caries/1033.txt  
  inflating: annotations/bboxes_caries/1042.txt  
  inflating: annotations/bboxes_caries/1050.txt  
  inflating: annotations/bboxes_caries/1058.txt  
  inflating: annotations/bboxes_caries/1067.txt  
  inflating: annotations/bboxes_caries/1074.txt  
  inflating: annotations/bboxes_caries/1080.txt  
  inflating: annotations/bboxes_caries/1083.txt  
  inflating: annotations/bboxes_caries/1088.txt  
  inflating: annotations/bboxes_caries/1090.txt  
  inflating: annotations/bboxes_caries/1091.txt  
  inflating: annotations/bboxes_caries/1092.txt  
  inflating: annotations/bboxes_caries/1096.txt  
  inflating: annotations/bboxes_caries/306.txt  
  inflating:

## 6. Carregamento do Conjunto de Dados

Verificamos o diretório onde os dados foram extraídos e listamos as classes (subpastas) existentes. Também podemos criar variáveis para armazenar o caminho principal dos dados.


In [ ]:

import os
import pandas as pd

data_dir = '/content/dataset'
classes = os.listdir(data_dir)
num_classes = len(classes)

## 7. Pré-Processamento de Dados

Nesta etapa, utilizamos o `ImageDataGenerator` para:
- Redimensionar os valores de pixel das imagens (dividindo por 255).
- Separar o dataset em treinamento e validação (usando `validation_split=0.2`).
- Ajustar o tamanho das imagens (150x150) e o batch size (32).

**Por que isso é importante?**
- Redimensionar (rescale) ajuda a normalizar os dados.
- Dividir em treino/validação permite monitorar se há overfitting.
- Padronizar tamanho de imagem facilita o processamento pela rede neural.


In [ ]:

from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

Found 320 images belonging to 6 classes.
Found 80 images belonging to 6 classes.


## 8. Construção do Modelo

Utilizamos uma rede neural **convolucional** simples, com:
- Camadas de `Conv2D` para extração de características.
- Camadas de `MaxPooling2D` para redução da dimensionalidade.
- `Flatten` para achatar a saída das convoluções.
- `Dense` para as camadas totalmente conectadas.
- `Dropout` para evitar overfitting.
- `softmax` na camada final, pois há mais de uma classe (multiclasse).

**Como isso melhora o modelo?**  
- As camadas convolucionais ajudam a detectar padrões em imagens (bordas, texturas).  
- O pooling reduz a complexidade, mantendo as informações mais relevantes.  
- O Dropout reduz o risco de o modelo decorar o conjunto de treinamento.


In [ ]:

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

## 9. Compilação do Modelo

Aqui definimos:
- Otimizador: `adam` (geralmente um bom ponto de partida).
- Função de perda: `categorical_crossentropy` (pois temos mais de 2 classes).
- Métrica principal: `accuracy`.

**Observação**: Poderíamos testar outros otimizadores ou ajustar a taxa de aprendizado para ver se obtemos melhores resultados.


In [ ]:

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

## 10. Treinamento do Modelo

Treinamos o modelo usando:
- `train_generator` para o conjunto de treino.
- `validation_generator` para o conjunto de validação.
- `epochs=10` (pode ajustar conforme necessidade).

**Melhorias**:
- Podemos incluir callbacks como `EarlyStopping` ou `ModelCheckpoint` para interromper o treinamento quando não houver melhora.
- Monitorar métricas como acurácia e perda ao longo das épocas.


In [ ]:

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=10
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 26s 2s/step - accuracy: 0.4592 - loss: 1.4368 - val_accuracy: 0.9375 - val_loss: 0.1957
Epoch 2/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 39s 2s/step - accuracy: 0.9387 - loss: 0.1709 - val_accuracy: 0.9844 - val_loss: 0.0675
Epoch 3/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 22s 2s/step - accuracy: 0.9767 - loss: 0.0639 - val_accuracy: 0.9844 - val_loss: 0.0380
Epoch 4/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 22s 2s/step - accuracy: 0.9834 - loss: 0.0570 - val_accuracy: 0.9844 - val_loss: 0.0321
Epoch 5/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 41s 2s/step - accuracy: 0.9906 - loss: 0.0215 - val_accuracy: 0.9844 - val_loss: 0.0398
Epoch 6/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 81s 7s/step - accuracy: 1.0000 - loss: 0.0099 - val_accuracy: 0.9844 - val_loss: 0.0425
Epoch 7/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 41s 2s/step - accuracy: 1.0000 - loss: 0.0066 - val_accuracy: 0.9844 - val_loss: 0.0298
Epoch 8/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 41s 2s/step - accuracy: 0.9966 - loss: 0.0081 - val_accuracy: 0.9844 - val_loss:

## 11. Avaliação do Modelo

Aqui avaliamos o modelo usando o `validation_generator`. Também podemos calcular a acurácia, que indica o percentual de acertos na classificação.


In [ ]:

loss, accuracy = model.evaluate(validation_generator)
print(f'Acurácia: {accuracy * 100:.2f}%')

3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 671ms/step - accuracy: 1.0000 - loss: 0.0144
Acurácia: 100.00%
